# X-ray Classification with TorchXRayVision

This notebook implements zero-shot classification on chest X-ray images using pretrained models from TorchXRayVision.

## Setup and Imports

In [ ]:
# Colab-specific setup (uncomment if using Colab)
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/Shareddrives/CS231N/chestxray-classification

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install torchxrayvision python-dotenv scikit-learn matplotlib tqdm --quiet

In [ ]:
# Standard imports
import os
import sys
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torchxrayvision as xrv
from PIL import Image

# Add the parent directory to path (if running in notebooks/)
sys.path.append('..')

In [ ]:
# Import custom modules
from utils.config import Config
from utils.data_utils import load_and_prepare_data
from utils.inference import run_inference
from models.xray_models import load_models
from evaluation.metrics import evaluate_predictions
from evaluation.visualization import plot_results, plot_roc_curves, plot_pr_curves

In [ ]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger("xray_evaluation")

## Configuration

In [ ]:
# Check if paths are set correctly
print(f"Data path: {Config.data_path}")
print(f"Image path: {Config.image_path}")
print(f"CSV path: {Config.csv_path}")
print(f"Output path: {Config.output_path}")
print(f"Using device: {Config.device}")
print(f"Target labels: {Config.target_labels}")

In [ ]:
# Update paths if needed
# Config.data_path = "/path/to/data"
# Config.image_path = os.path.join(Config.data_path, "images")
# Config.csv_path = os.path.join(Config.data_path, "Data_Entry_2017_v2020.csv")

# Create output directory
os.makedirs(Config.output_path, exist_ok=True)

## Load Models

In [ ]:
# List available models
print("Available pretrained models:")
for model_name in xrv.models.model_urls.keys():
    print(f"- {model_name}")

In [ ]:
# Load models
model_mimic, model_chex, pathologies = load_models(Config.device)

In [ ]:
# Display pathologies
print("\nSupported pathologies:")
for i, pathology in enumerate(pathologies):
    print(f"{i}: {pathology}")

# Highlight target pathologies
print("\nTarget pathologies:")
for label in Config.target_labels:
    idx = pathologies.index(label)
    print(f"{idx}: {label}")

## Load and Prepare Data

In [ ]:
# Load and prepare data
test_df, label_indices = load_and_prepare_data(Config, pathologies)

In [ ]:
# Display dataset composition
print("\nDataset Composition:")
for label in Config.target_labels:
    count = test_df[label].sum()
    print(f"{label}: {count} positive samples")

# Display negative cases
negative_count = (test_df[Config.target_labels].sum(axis=1) == 0).sum()
print(f"No target diseases: {negative_count} samples")

In [ ]:
# Visualize class distribution
plt.figure(figsize=(10, 5))
test_df[Config.target_labels].sum().plot(kind="bar", color="lightgreen")
plt.title("Positive Case Count per Disease")
plt.ylabel("Count")
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Visualize Sample Images

In [ ]:
# Function to display sample X-ray images
def display_sample_images(test_df, image_path, num_samples=3):
    """Display sample X-ray images for each disease category"""
    fig, axes = plt.subplots(len(Config.target_labels) + 1, num_samples, 
                            figsize=(num_samples * 4, (len(Config.target_labels) + 1) * 4))
    
    # For each disease, show samples
    for i, disease in enumerate(Config.target_labels):
        disease_samples = test_df[test_df[disease] == 1].sample(num_samples)
        
        for j, (_, row) in enumerate(disease_samples.iterrows()):
            img_file = os.path.join(image_path, row["Image Index"])
            img = Image.open(img_file).convert('L')
            axes[i, j].imshow(img, cmap='gray')
            axes[i, j].set_title(f"{disease}\n{row['Image Index']}")
            axes[i, j].axis('off')
    
    # Show negative samples (no target disease)
    negative_samples = test_df[(test_df[Config.target_labels] == 0).all(axis=1)].sample(num_samples)
    
    for j, (_, row) in enumerate(negative_samples.iterrows()):
        img_file = os.path.join(image_path, row["Image Index"])
        img = Image.open(img_file).convert('L')
        axes[-1, j].imshow(img, cmap='gray')
        axes[-1, j].set_title(f"No Target Disease\n{row['Image Index']}")
        axes[-1, j].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Display sample images
display_sample_images(test_df, Config.image_path, num_samples=3)

## Run Inference

In [ ]:
# Run inference
y_true, y_pred = run_inference(test_df, model_mimic, model_chex, label_indices, Config)

## Evaluate Results

In [ ]:
# Evaluate predictions
results_df = evaluate_predictions(y_true, y_pred, Config.target_labels, Config.output_path)

In [ ]:
# Display results dataframe
results_df

## Visualize Results

In [ ]:
# Plot summary results
plot_results(results_df, test_df, Config.target_labels, Config.output_path)

In [ ]:
# Plot ROC curves
plot_roc_curves(y_true, y_pred, Config.target_labels, Config.output_path)

In [ ]:
# Plot Precision-Recall curves
plot_pr_curves(y_true, y_pred, Config.target_labels, Config.output_path)

## Conclusion

Summary of findings and next steps.